In [1]:
import sys, os, math
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '../../quizx/pybindings')
import pyzx as zx
import quizx as qzx
import numpy as np
from pyzx import print_matrix
from pyzx.basicrules import *
import time
from fractions import Fraction

Z = zx.VertexType.Z
X = zx.VertexType.X
B = zx.VertexType.BOUNDARY
SE = zx.EdgeType.SIMPLE
HE = zx.EdgeType.HADAMARD

In [29]:
c = zx.qasm("""
qreg q[4];
cx q[0], q[1];
cx q[2], q[1];
cx q[2], q[3];
cx q[3], q[0];
cx q[3], q[1];
cx q[2], q[1];
cx q[1], q[2];
""")

In [27]:
fuse(g, 7, 9)

KeyError: 7

In [28]:
zx.draw(g)

In [30]:
g = c.to_graph()
while any(fuse(g, g.edge_s(e), g.edge_t(e)) for e in g.edges()): pass
zx.draw(g)

In [31]:
def finished(g, v):
    return ((g.type(v) == X) and any(w in g.outputs() for w in g.neighbors(v)) or
            (g.type(v) == Z) and any(w in g.inputs()  for w in g.neighbors(v)))

def strong_comp_safe(g, v0, v1):
    if finished(g, v0) or finished(g, v1): return False
    else: return strong_comp(g, v0, v1)

def phase_free_simp(g):
    while (
        any(fuse(g, g.edge_s(e), g.edge_t(e)) for e in g.edges()) or
        any(strong_comp_safe(g, g.edge_s(e), g.edge_t(e)) for e in g.edges())
    ): pass

In [32]:
g = c.to_graph()
phase_free_simp(g)
zx.draw(g)

In [15]:
c = zx.generate.CNOT_HAD_PHASE_circuit(qubits=5, depth=400, clifford=True)
g = c.to_graph()

In [16]:
zx.draw(g)

In [18]:
zx.simplify.clifford_simp(g)
g.normalize()
zx.draw(g)

In [19]:
c1 = zx.extract_circuit(g.copy(), up_to_perm=True)
zx.draw(c1)

In [20]:
g = c.to_graph(backend="quizx-vec")

In [21]:
zx.draw(g)

In [22]:
gs = g.copy()
%time zx.simplify.clifford_simp(gs)
gs.normalize()
zx.draw(gs)

CPU times: user 345 ms, sys: 1.76 ms, total: 347 ms
Wall time: 350 ms


In [23]:
gs = g.copy()
%time qzx.simplify.clifford_simp(gs)
gs.normalize()
zx.draw(gs)

CPU times: user 2.72 ms, sys: 0 ns, total: 2.72 ms
Wall time: 2.72 ms


In [37]:
n = 50
g = 30
pfirst = 1/n
psecond = (1-pfirst)*1/(n-1)
pthird = (1-pfirst)*(1-psecond)*(1/(n-2))
p0 = pfirst + psecond + pthird
p = 1
for i in range(g): p *= 1-p0
p = 1-p
p

0.8437859458338086